In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
import time
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/TFG/Dictionary

/content/drive/MyDrive/Colab Notebooks/TFG/Dictionary


In [2]:
def definizioak_adibideak_lortu(hitza, fichero):
  try:
    URL = 'https://en.oxforddictionaries.com/definition/' + hitza
    html = urlopen(URL).read().decode('utf-8')
  except HTTPError:
    print('Error al entrar en la pagina web de la palabra {}'.format(hitza))
    return {}

  try:
    html_info = BeautifulSoup(html, "lxml") #arbol con la informacion del html
    for div in html_info.find_all("section", {'class':'etymology'}):
      div.decompose() #eliminamos del arbol la informacion que no nos interesa

    sekzioak = html_info.find_all(attrs={"class": "gramb"}) #sekzioen lista (sekzio: NOUN, ADVERB, ADJECTIVE...)

    i = 1 #definizioen indizea
    adb_kopurua = 0

    for s in sekzioak:
      #######################################################################
      # Busca el tipo de palabra (NOUN / VERB / ...) y la palabra en plural, el infinitivo...
      #######################################################################  
      seccion_aux = s.find_all(attrs={"class": "ps"})[0]
      secciones = seccion_aux.find_all(attrs={"class": "listSeparator"})

      if len(secciones) != 0:
        for div in html_info.find_all("span", {'class':'listSeparator'}):
          div.decompose()

      seccion = re.split('<span class="pos">(.*?)</span>', str(seccion_aux)) #NOUN / VERB / ADJECTIVE /...

      derivados = re.findall('<span>(.*?)</span>', str(seccion_aux)) #la palabra en plural, el infinitivo...
      if len(derivados) != 0:
        lista_derivados = []
        for der in derivados:
          lista_derivados.append(der)
        deriv = " ".join(lista_derivados)

      #######################################################################
      # Busca e imprime la transitividad ([NO OBJECT] / [WITH OBJECT])
      #######################################################################
      transitividad_aux = s.find_all(attrs={"class": "transitivity"})
      transitividad = re.split('<span class="neutral">(.)</span>(.*?)<span class="neutral">(.)</span>', str(transitividad_aux))

      if len(transitividad) > 1:
        if len(derivados) != 0:
          print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + '(' +  deriv + ') ' + transitividad[1] + transitividad[2] + transitividad[3])
          fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + '(' +  deriv + ') ' + transitividad[1] + transitividad[2] + transitividad[3] + '\n')
        else:
          print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + transitividad[1] + transitividad[2] + transitividad[3])
          fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + transitividad[1] + transitividad[2] + transitividad[3] + '\n')
      else:
        if len(derivados) != 0:
          print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + '(' +  deriv + ') ')
          fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + '(' +  deriv + ')\n')
      
      #######################################################################
      # Busca e imprime el dominio (Chemistry / Geology / ...)
      #######################################################################
      dominio_aux = s.find_all(attrs={"class": "domain_labels"})
      if len(dominio_aux) > 0:
        dominio = re.split('<span class="domain_labels">(.*?)</span>', str(dominio_aux[0]))

        for d in dominio:
          for a in range(len(d)):
            if d[a] is '>':
              principio = a
            elif d[a] is '/':
              fin = a
              break

          lista_dominio = []
              
          for c in range(principio+1, fin-1):
            lista_dominio.append(d[c])

          dom = "".join(lista_dominio)

          print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DOM: {}\n'.format(dom))
          fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DOM: {}\n'.format(dom))


      #######################################################################
      #Busca e imprime todas las definiciones con sus ejemplos y sinonimos
      #######################################################################
      prueba = s.find_all(attrs={"class": "semb"}) #para saber si esa seccion tiene definiciones o no
      
      if len(prueba) != 0: #si la seccion tiene informacion
        defi_adib_guztiak_aux = prueba[0]
        defi_adib_guztiak = defi_adib_guztiak_aux.find_all('li', "") #sekzio bakoitzeko definizioak
      
        for dag in defi_adib_guztiak: #por cada definicion    
          
          if str(dag)[:4] == '<li>':
            ind = dag.find_all(attrs={"class": "ind"})
            
            if len(ind) > 0: #si tiene una frase como definicion
              defi = re.split('<span class="ind">(.*?)</span>', str(ind))
            
              for v in defi:
                for h in range(len(v)):
                  if v[h] is '>':
                    principio = h
                  elif v[h] is '/':
                    fin = h
                    break
              
                lista_definicion = []
              
                for c in range(principio+1, fin-1):
                  lista_definicion.append(v[c])

                definicion = "".join(lista_definicion)
                print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + definicion)
                fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + definicion + '\n')

            else: #si tiene una referencia como definicion
              
              referencia = dag.find_all(attrs={"class": "crossReference"})
              ref = re.split('<a href="/definition/(.*?)">(.*?)</a>', str(referencia))

              print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}'.format(ref[1]))
              fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\n'.format(ref[1]))
              

            defi_adib = dag.find_all(attrs={"class": "trg"}) #lista honetako elementu bakoitza definizio bat da bere adibideekin
            defi_adib_string = str(defi_adib) #antes ponia defi_adib[0]
            definizioak = re.split('<span class="ind">(.*?)</span>', defi_adib_string) #lista honetako elementu bakoitza definizio bat da

            for d in definizioak:

              subdefiniciones = dag.find_all(attrs={"class": "subSenses"})

              hay_ejemplos = True
              if len(subdefiniciones) != 0: #si tiene subdefiniciones

                buscar = re.split('<div class="exg">(.*?)<ol class="subSenses">', d)
                if len(buscar) > 1: #la definicion tiene ejemplos
                  adibideak = re.findall('<em>(.*?)</em>', buscar[1])
                else:
                  hay_ejemplos = False

              else:    
                adibideak = re.findall('<em>(.*?)</em>', d) #adibide esaldien lista

              if hay_ejemplos and len(adibideak) != 0:

                j = 1 #adibideen indizea
                for a in adibideak:

                  primer_ejemplo = False
                  for h in range(len(a)):
                    if a[h] is '>':
                      principio = h
                      primer_ejemplo = True
                    elif a[h] is '/':
                      fin = h
                      break
              
                  if primer_ejemplo is True:
                    lista_ejemplo = []
              
                    for c in range(principio+1, fin-1):
                      lista_ejemplo.append(a[c])

                    ejemplo = "".join(lista_ejemplo)
                    print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'EX: {} \t'.format(j) + ejemplo)
                    fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'EX: {} \t'.format(j) + ejemplo + '\n')

                  elif j<10:
                    print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'EX: {} \t'.format(j) + a)
                    fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'EX: {} \t'.format(j) + a + '\n')
                  else:
                    print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'EX: {}\t'.format(j) + a)
                    fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'EX: {}\t'.format(j) + a + '\n')
                  j +=1

                adb_kopurua += j-1

              sinonimo1 = re.split('<strong class="syn">(.*?)</strong>', defi_adib_string)
              if len(sinonimo1) > 1:
                sinonimos = re.split('<span class="syn">(.*?)</span>', defi_adib_string)
                if len(sinonimos) > 1:
                  print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SYN: ' + sinonimo1[1] + sinonimos[1] + '\n')
                  fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SYN: ' + sinonimo1[1] + sinonimos[1] + '\n')
                else:
                  print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SYN: ' + sinonimo1[1] + '\n')
                  fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SYN: ' + sinonimo1[1] + '\n')
              fichero.write('\n')
            
            #######################################################################
            # Busca e imprime todas las subdefiniciones con sus ejemplos y sinonimos
            #######################################################################
            subdefiniciones = dag.find_all(attrs={"class": "subSense"})
            
            if len(subdefiniciones) != 0: #si la definicion tiene subdefiniciones
              k = 1 #indice subdefiniciones

              for subdefs in subdefiniciones: #por cada subdefinicion
                
                subdefini = subdefs.find_all(attrs={"class": "ind"})

                if len(subdefini) > 0: #si tiene una frase como subdefinicion
                  subdefi = re.split('<span class="one-click-content">(.*?)</span>', str(subdefini))

                  for v in subdefi:

                    for h in range(len(v)):
                      if v[h] is '>':
                        principio = h
                      elif v[h] is '/':
                        fin = h
                        break
                  
                    lista_subdefinicion = []
                    for u in range(principio+1, fin-1):
                      lista_subdefinicion.append(v[u])

                    subdefinicion = "".join(lista_subdefinicion)
                    print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + subdefinicion)
                    fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + subdefinicion + '\n')
                
                else: #si tiene una referencia como subdefinicion

                  referencia = subdefs.find_all(attrs={"class": "crossReference"})
                  ref = re.split('<a href="/definition/(.*?)">(.*?)</a>', str(referencia))

                  print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + ref[1])
                  fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + ref[1] + '\n')
                
                subdef_todas = subdefiniciones[0]
                subdef_todas_string = str(subdef_todas)
                
                subejem_aux = subdefs.find_all(attrs={"class": "exg"})
                subejem = re.split('<span class="ex">(.*?)</span>', str(subejem_aux))

                for se in subejem:

                  subejemplos = re.findall('<em>(.*?)</em>', se)

                  if len(subejemplos) != 0:

                    j = 1 #indice ejemplos
                    for e in subejemplos:

                      primer_ejemplo = False
                      for h in range(len(e)):
                        if e[h] is '>':
                          principio = h
                          primer_ejemplo = True
                        elif e[h] is '/':
                          fin = h
                          break
              
                      if primer_ejemplo is True:
                        lista_subejemplo = []
              
                        for c in range(principio+1, fin-1):
                          lista_subejemplo.append(e[c])

                        subejemplo = "".join(lista_subejemplo)
                        print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'EX: {} \t'.format(j) + subejemplo)
                        fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'EX: {} \t'.format(j) + subejemplo + '\n')
                      
                      elif j<10:
                        print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'EX: {} \t'.format(j) + e)
                        fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'EX: {} \t'.format(j) + e + '\n')
                      else:
                        print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'EX: {}\t'.format(j) + e)
                        fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'EX: {}\t'.format(j) + e + '\n')
                      j +=1

                    adb_kopurua += j-1
                 
                  sinonimo1 = re.findall('<strong class="syn">(.*?)</strong>', subdef_todas_string)
                  if len(sinonimo1) > 1:
                    sinonimos = re.split('<span class="syn">(.*?)</span>', subdef_todas_string)
                    if len(sinonimos) > 1:
                      print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'SYN: ' + sinonimo1[1] + sinonimos[1])
                      fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'SYN: ' + sinonimo1[1] + sinonimos[1] + '\n')
                    else:
                      print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'SYN: ' + sinonimo1[1])
                      fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'DEF: {}\t'.format(i) + 'SUBDEF: {}\t'.format(k) + 'SYN: ' + sinonimo1[1] + '\n')

                  k +=1
                  print('\n')
                  fichero.write('\n')

            i +=1

      else: #si la seccion no tiene informacion
        
        sin_definiciones = s.find_all(attrs={"class": "empty_sense"})

        if len(sin_definiciones) != 0:

          ejemplos_impresos = False
          sinonimos_impresos = False

          sin_definiciones = sin_definiciones[0]
          referencia = sin_definiciones.find_all(attrs={"class": "derivative_of"})
          
          if len(referencia) > 0:
            r = re.split('<a href="(.*?)">(.*?)</a>', str(referencia))
           
            ejemplos = re.findall('<em>(.*?)</em>', str(sin_definiciones))

            if len(ejemplos) != 0: #si tiene ejemplos
              ejemplos_impresos = True
              j=1
              for e in ejemplos:

                primer_ejemplo = False
                for h in range(len(e)):
                  if e[h] is '>':
                    principio = h
                    primer_ejemplo = True
                  elif e[h] is '/':
                    fin = h
                    break
              
                if primer_ejemplo is True:  
                  lista_ejemplo = []
              
                  for c in range(principio+1, fin-1):
                    lista_ejemplo.append(e[c])

                  ejemplo = "".join(lista_ejemplo)
                  print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'EX: {}\t'.format(j) + ejemplo)
                  fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'EX: {}\t'.format(j) + ejemplo + '\n')
                
                else:
                  print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'EX: {}\t'.format(j) + e)
                  fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'EX: {}\t'.format(j) + e + '\n')

                j +=1
              adb_kopurua += j-1

            else: #si no tiene ejemplos
              print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]))
              fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\n'.format(r[1]))
            
            sinonimo1 = re.split('<strong class="syn">(.*?)</strong>', str(sin_definiciones))
            if len(sinonimo1) > 1:
              sinonimos_impresos = True
              sinonimos = re.split('<span class="syn">(.*?)</span>', str(sin_definiciones))
              if len(sinonimos) > 1:
                print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'SYN: ' + sinonimo1[1] + sinonimos[1] + '\n')
                fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'SYN: ' + sinonimo1[1] + sinonimos[1] + '\n')
              else:
                print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'SYN: ' + sinonimo1[1] + '\n')
                fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\t'.format(r[1]) + 'SYN: ' + sinonimo1[1] + '\n')
            fichero.write('\n')         

          referencia_otra = sin_definiciones.find_all(attrs={"class": "crossReference"})
          
          if len(referencia_otra) > 0:
            ref = re.split('<a href="/definition/(.*?)">(.*?)</a>', str(referencia_otra))

            print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}'.format(ref[2]))
            fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'REF: {}\n'.format(ref[2]))

          if ejemplos_impresos is False: #si no tiene definicion ni referencia pero si ejemplos

            ejemplos = re.findall('<em>(.*?)</em>', str(sin_definiciones))

            if len(ejemplos) != 0: #si tiene ejemplos
              j=1
              for e in ejemplos:

                
                primer_ejemplo = False
                for h in range(len(e)):
                  if e[h] is '>':
                    principio = h
                    primer_ejemplo = True
                  elif e[h] is '/':
                    fin = h
                    break
              
                if primer_ejemplo is True:  
                  lista_ejemplo = []
              
                  for c in range(principio+1, fin-1):
                    lista_ejemplo.append(e[c])

                  ejemplo = "".join(lista_ejemplo)
                  print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'EX: {}\t'.format(j) + ejemplo)
                  fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'EX: {}\t'.format(j) + ejemplo + '\n')
                
                
                else:
                  print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'EX: {}\t'.format(j) + e)
                  fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'EX: {}\t'.format(j) + e + '\n')

                j +=1
              adb_kopurua += j-1

          if sinonimos_impresos is False: #si no tiene definicion ni referencia pero si sinonimos

            sinonimo1 = re.split('<strong class="syn">(.*?)</strong>', str(sin_definiciones))
            if len(sinonimo1) > 1:
              sinonimos = re.split('<span class="syn">(.*?)</span>', str(sin_definiciones))
              if len(sinonimos) > 1:
                print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'SYN: ' + sinonimo1[1] + sinonimos[1] + '\n')
                fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'SYN: ' + sinonimo1[1] + sinonimos[1] + '\n')
              else:
                print('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'SYN: ' + sinonimo1[1] + '\n')
                fichero.write('WORD: {}\t'.format(hitza)+ 'POS: {}\t'.format(seccion[1]) + 'SYN: ' + sinonimo1[1] + '\n')
            fichero.write('\n')
            

    #Imprime la cantidad de definiciones y ejemplos que tiene la palabra
    i -=1
    print('\nTOTAL AMOUNT OF DEFINITIONS OF THE WORD {}: '.format(hitza), end="      ")
    print(i)
    print('TOTAL AMOUNT OF EXAMPLE SENTENCES OF THE WORD {}: '.format(hitza), end=" ")
    print(adb_kopurua)

  except:
    print('An error occured in the function definizioak_adibideak_lortu with the word {}'.format(hitza))
    fichero.write('An error occured in the function definizioak_adibideak_lortu with the word {}\n\n'.format(hitza))
    return {}

In [3]:
fichero = open("oneword.txt", "w")
definizioak_adibideak_lortu('hospital', fichero)
fichero.close()

WORD: hospital	POS: noun	DEF: 1	An institution providing medical and surgical treatment and nursing care for sick or injured people.
WORD: hospital	POS: noun	DEF: 1	EX: 1 	‘My doctor has referred me to the eye clinic at the local hospital for surgical treatment.’
WORD: hospital	POS: noun	DEF: 1	EX: 2 	‘The medical wards of hospitals admit the oldest and sickest people in our community.’
WORD: hospital	POS: noun	DEF: 1	EX: 3 	‘Neath is a smaller hospital with a busy medical intake but no acute surgical services.’
WORD: hospital	POS: noun	DEF: 1	EX: 4 	‘The injured were still undergoing intensive care at two hospitals in the city.’
WORD: hospital	POS: noun	DEF: 1	EX: 5 	‘For adult critical care, star ratings do not reflect the quality of clinical care provided by hospitals.’
WORD: hospital	POS: noun	DEF: 1	EX: 6 	‘However, trying to get this information from primary care trusts or hospitals is very difficult.’
WORD: hospital	POS: noun	DEF: 1	EX: 7 	‘Evidence also exists that the quality 

In [ ]:
def hiztegia_lortu_fichero(f_read, f_write):
  try:
    for line in f_read:
      for part in line.split():
        if "WORD:" in part: #PAGE  and (line[16] is '1' or line[16] is '2') and line[17] is not ' ' #NUMBER and line[27] is '2' and line[29] is not ' '
          print(line)
          f_write.write(line)
          end = len(line) - 1
          if line[17] is ' ': #PAGE <10
            lista_palabra = list(line[36:end])
          elif line[18] is ' ': #10<=  PAGE <100
            lista_palabra = list(line[37:end])
          else: #PAGE >=100
            lista_palabra = list(line[38:end])

          #la palabra no puede contener espacios en blanco ni tildes u otros caracteres especiales
          for idx in range(len(lista_palabra)):

            letra = lista_palabra[idx]

            if letra is ' ':
              lista_palabra[idx] = '_'
            elif letra is 'ä' or letra is 'á' or letra is 'à' or letra is 'â' or letra is 'ã' or letra is 'Á':
              lista_palabra[idx] = 'a'
            elif letra is 'ë' or letra is 'é' or letra is 'è' or letra is 'ê' or letra is 'ẽ' or letra is 'É' or letra is 'ę':
              lista_palabra[idx] = 'e'
            elif letra is 'ï' or letra is 'í' or letra is 'ì' or letra is 'î' or letra is 'ĩ' or letra is 'Í':
              lista_palabra[idx] = 'i'
            elif letra is 'ö' or letra is 'ó' or letra is 'ò' or letra is 'ô' or letra is 'õ' or letra is 'Ó' or letra is 'ø' or letra is 'Ø':
              lista_palabra[idx] = 'o'
            elif letra is 'ü' or letra is 'ú' or letra is 'ù' or letra is 'û' or letra is 'ũ' or letra is 'Ú':
              lista_palabra[idx] = 'u'
            elif letra is 'ñ':
              lista_palabra[idx] = 'n'
            elif letra is 'Č' or letra is 'č':
              lista_palabra[idx] = 'c'
            elif letra is 'š':
              lista_palabra[idx] = 's'
            elif letra is '&': 
              for idx1 in range(idx+1, idx+5): # &amp;
                lista_palabra[idx1] = ' '


          palabra = "".join(lista_palabra)
      
          try:
            definizioak_adibideak_lortu(palabra, f_write)
            f_write.write('\n')
          except:
            print('ops! could not call the function definizioak_adibideak_lortu with the word {}'.format(palabra))
            f_write.write('ops! could not call the function definizioak_adibideak_lortu with the word {}\n\n'.format(palabra))

          #entre palabra y palabra, esperar entre 1 y 2 segundos
          wait_time = random.randint(1, 2)
          time.sleep(wait_time)


  except:
    print('Something failed in the function hiztegia_lortu_fichero reading from the file {}'.format(f_read))
    f_write.write('Something failed in the function hiztegia_lortu_fichero reading from the file {}\n\n'.format(f_read))
    return {}

In [ ]:
def hiztegi_osoa_lortu():
  fileR = open("/content/drive/MyDrive/Colab Notebooks/TFG/Wordlists/#_wordlist.txt", "r") #"r" si queremos leer
  fileW = open("#_dictionary.txt", "w") #"w" si queremos sobreescribir, "a" si queremos escribir a continuacion del contenido que ya tenga
  hiztegia_lortu_fichero(fileR, fileW)
  fileR.close()
  fileW.close()

In [ ]:
hiztegi_osoa_lortu()

Se han truncado las últimas 5000 líneas del flujo de salida.
ops! could not call the function definizioak_adibideak_lortu with the word ¥
LETTER: 0 PAGE: 1 NUMBER: 21  WORD: ≤

ops! could not call the function definizioak_adibideak_lortu with the word ≤
LETTER: 0 PAGE: 1 NUMBER: 22  WORD: ♂

ops! could not call the function definizioak_adibideak_lortu with the word ♂
LETTER: 0 PAGE: 1 NUMBER: 23  WORD: ♯

ops! could not call the function definizioak_adibideak_lortu with the word ♯
LETTER: 0 PAGE: 1 NUMBER: 24  WORD: √

ops! could not call the function definizioak_adibideak_lortu with the word √
LETTER: 0 PAGE: 1 NUMBER: 25  WORD: ©

ops! could not call the function definizioak_adibideak_lortu with the word ©
LETTER: 0 PAGE: 1 NUMBER: 26  WORD: ⁂

ops! could not call the function definizioak_adibideak_lortu with the word ⁂
LETTER: 0 PAGE: 1 NUMBER: 27  WORD: ∞

ops! could not call the function definizioak_adibideak_lortu with the word ∞
LETTER: 0 PAGE: 1 NUMBER: 28  WORD: ♭

ops! could 